In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import argparse
import os
import torch
import pickle
from functools import partial

from transformer_lens import HookedTransformer, HookedTransformerConfig
from huggingface_hub import hf_hub_download

from MIB_circuit_track.dataset import HFEAPDataset
from eap.graph import Graph
from eap.attribute import attribute
from eap.attribute_node import attribute_node
from MIB_circuit_track.metrics import get_metric
from MIB_circuit_track.utils import MODEL_NAME_TO_FULLNAME, TASKS_TO_HF_NAMES, COL_MAPPING

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model_name = "gpt2"
model = HookedTransformer.from_pretrained(model_name)
model.cfg.use_split_qkv_input = True
model.cfg.use_attn_result = True
model.cfg.use_hook_mlp_in = True
model.cfg.ungroup_grouped_query_attention = True

task = 'ioi'
graph = Graph.from_model(model, neuron_level=True, node_scores=False)

hf_task_name = f'mib-bench/{TASKS_TO_HF_NAMES[task]}'
dataset = HFEAPDataset(hf_task_name, model.tokenizer, split="validation", task=task, model_name=model_name, num_examples=1)

dataloader = dataset.to_dataloader(batch_size=1)
metric = get_metric('logit_diff', task, model.tokenizer, model)
attribution_metric = partial(metric, mean=True, loss=True)


Loaded pretrained model gpt2 into HookedTransformer


In [8]:
attribute_node(model, graph, dataloader, attribution_metric, "EAP", 
                    "patching", neuron=True, ig_steps=5,
                    optimal_ablation_path=None,
                    intervention_dataloader=dataloader)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [45]:
len(graph.nodes), graph.neurons_scores.shape, graph.neurons_in_graph.shape, graph.nodes_in_graph.shape

(158, torch.Size([157, 768]), torch.Size([157, 768]), torch.Size([157]))

In [27]:
from eap.evaluate import evaluate_graph, evaluate_baseline

baseline = evaluate_baseline(model, dataloader, attribution_metric).mean().item()
results = evaluate_graph(model, graph, dataloader, attribution_metric).mean().item()
print(f"Original performance was {baseline}; the circuit's performance is {results}")

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 10.42it/s]


Original performance was -3.20346736907959; the circuit's performance is 3.045830726623535


In [46]:
for node_name, node in graph.nodes.items():
    print(f"{node_name}: {graph.forward_index(node)}")

input: 0
a0.h0: slice(1, 13, None)
a0.h1: slice(1, 13, None)
a0.h2: slice(1, 13, None)
a0.h3: slice(1, 13, None)
a0.h4: slice(1, 13, None)
a0.h5: slice(1, 13, None)
a0.h6: slice(1, 13, None)
a0.h7: slice(1, 13, None)
a0.h8: slice(1, 13, None)
a0.h9: slice(1, 13, None)
a0.h10: slice(1, 13, None)
a0.h11: slice(1, 13, None)
m0: 13
a1.h0: slice(14, 26, None)
a1.h1: slice(14, 26, None)
a1.h2: slice(14, 26, None)
a1.h3: slice(14, 26, None)
a1.h4: slice(14, 26, None)
a1.h5: slice(14, 26, None)
a1.h6: slice(14, 26, None)
a1.h7: slice(14, 26, None)
a1.h8: slice(14, 26, None)
a1.h9: slice(14, 26, None)
a1.h10: slice(14, 26, None)
a1.h11: slice(14, 26, None)
m1: 26
a2.h0: slice(27, 39, None)
a2.h1: slice(27, 39, None)
a2.h2: slice(27, 39, None)
a2.h3: slice(27, 39, None)
a2.h4: slice(27, 39, None)
a2.h5: slice(27, 39, None)
a2.h6: slice(27, 39, None)
a2.h7: slice(27, 39, None)
a2.h8: slice(27, 39, None)
a2.h9: slice(27, 39, None)
a2.h10: slice(27, 39, None)
a2.h11: slice(27, 39, None)
m2: 39
a3.h

In [49]:
graph.to_json(f'circuits/debug/importances.json')

Node input in graph: False
Node a0.h0 in graph: False
Node a0.h1 in graph: False
Node a0.h2 in graph: False
Node a0.h3 in graph: False
Node a0.h4 in graph: False
Node a0.h5 in graph: False
Node a0.h6 in graph: False
Node a0.h7 in graph: False
Node a0.h8 in graph: False
Node a0.h9 in graph: False
Node a0.h10 in graph: False
Node a0.h11 in graph: False
Node m0 in graph: False
Node a1.h0 in graph: False
Node a1.h1 in graph: False
Node a1.h2 in graph: False
Node a1.h3 in graph: False
Node a1.h4 in graph: False
Node a1.h5 in graph: False
Node a1.h6 in graph: False
Node a1.h7 in graph: False
Node a1.h8 in graph: False
Node a1.h9 in graph: False
Node a1.h10 in graph: False
Node a1.h11 in graph: False
Node m1 in graph: False
Node a2.h0 in graph: False
Node a2.h1 in graph: False
Node a2.h2 in graph: False
Node a2.h3 in graph: False
Node a2.h4 in graph: False
Node a2.h5 in graph: False
Node a2.h6 in graph: False
Node a2.h7 in graph: False
Node a2.h8 in graph: False
Node a2.h9 in graph: False
Nod

In [51]:
graph = Graph.from_json('circuits/debug/importances.json')

In [52]:
from eap.evaluate import evaluate_graph, evaluate_baseline

baseline = evaluate_baseline(model, dataloader, attribution_metric).mean().item()
results = evaluate_graph(model, graph, dataloader, attribution_metric).mean().item()
print(f"Original performance was {baseline}; the circuit's performance is {results}")

100%|██████████| 1/1 [00:00<00:00, 12.37it/s]

Original performance was -3.20346736907959; the circuit's performance is 3.045830726623535


In [1]:
import pickle

with open('results/EAP_patching_neuron/ioi_gpt2_validation_abs-False.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
data

{'weighted_edge_counts': [3.28125,
  9.4140625,
  31.35416603088379,
  87.86458587646484,
  291.95703125,
  1420.6614990234375,
  5055.1484375,
  13690.2861328125,
  34932.48828125,
  69865.0],
 'area_under': 0.25095041562974435,
 'area_from_1': 0.7480495843702557,
 'average': 0.10142104345053557,
 'faithfulnesses': [0.0,
  0.0,
  0.0,
  -1.4140530751639912e-05,
  6.898501866689867e-05,
  0.0011000054684707906,
  0.0085503075944916,
  0.00769121043382629,
  -0.0031859334793482943,
  1.0]}

### testing custom

In [13]:
from custom_method import custom_attribute_node, get_scores_ig_activations_directional, asymmetry_score

In [17]:
corrupt_to_clean = get_scores_ig_activations_directional(
    model, graph, dataloader, metric, intervention='patching', 
    intervention_dataloader=dataloader, 
    optimal_ablation_path=None, quiet=False,
    neuron=True, patch_direction='patch-in-corrupt'
)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [01:09<00:00, 69.53s/it]


In [18]:
clean_to_corrupt = get_scores_ig_activations_directional(
        model, graph, dataloader, metric, intervention='patching', 
        intervention_dataloader=dataloader, 
        optimal_ablation_path=None, quiet=False,
        neuron=True, patch_direction='patch-in-clean')

100%|██████████| 1/1 [01:09<00:00, 69.81s/it]


In [21]:
corrupt_to_clean.shape, clean_to_corrupt.shape

(torch.Size([157, 768]), torch.Size([157, 768]))

In [29]:
scores_asymmetry = asymmetry_score(corrupt_to_clean, clean_to_corrupt)
abs_scores_asymmetry = scores_asymmetry.abs()
threshold = torch.quantile(abs_scores_asymmetry.flatten(), 0.9)

latent_components = abs_scores_asymmetry >= threshold
latent_components_indices = latent_components.nonzero()

In [31]:
scores = corrupt_to_clean.clone()

for n, d in latent_components_indices:
    scores[n, d] = clean_to_corrupt[n, d]

In [32]:
custom_attribute_node(model, graph, dataloader, attribution_metric, "patching", neuron=True, intervention_dataloader=dataloader)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [02:02<00:00, 122.25s/it]


In [33]:
from eap.evaluate import evaluate_graph, evaluate_baseline

baseline = evaluate_baseline(model, dataloader, attribution_metric).mean().item()
results = evaluate_graph(model, graph, dataloader, attribution_metric).mean().item()
print(f"Original performance was {baseline}; the circuit's performance is {results}")

100%|██████████| 1/1 [00:00<00:00,  9.80it/s]

Original performance was -3.20346736907959; the circuit's performance is 3.045830726623535


In [34]:
print(f"Faithfulness: {results / baseline}. Original {baseline}, new {results}")

Faithfulness: -0.9507918688426202. Original -3.20346736907959, new 3.045830726623535


In [ ]:
from MIB_circuit_track.evaluation import evaluate_area_under_curve

eval_auc_outputs = evaluate_area_under_curve(model, graph, dataloader, attribution_metric, level='neuron')
weighted_edge_counts, area_under, area_from_1, average, faithfulnesses = eval_auc_outputs

print(f"weighted_edge_counts: {weighted_edge_counts}")
print(f"area_under (cpr): {area_under}") # higher is better
print(f"area_from_1 (cmd): {area_from_1}") # lower is better
print(f"average: {average}")
print(f"faithfulnesses: {faithfulnesses}")

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 11.31it/s]


Computing results for 0.1% of neurons (N=120)


100%|██████████| 1/1 [00:00<00:00,  9.72it/s]


Computing results for 0.2% of neurons (N=241)


...

Computing results for 50.0% of neurons (N=60288)


100%|██████████| 1/1 [00:00<00:00, 11.32it/s]


Computing results for 100% of neurons (N=120576)


100%|██████████| 1/1 [00:00<00:00,  9.62it/s]

weighted_edge_counts: [1.25, 2.5104167461395264, 22.73177146911621, 95.70963287353516, 323.3502502441406, 1373.5677490234375, 4160.29296875, 12088.8994140625, 34932.50390625, 69865.0]
area_under (cpr): 0.8379132548760359
area_from_1 (cmd): 0.17193037014044543
average: 0.32901708321735346
faithfulnesses: [-1.5260502888507987e-07, 1.5260502888507987e-07, -0.0019821867201883026, 0.003414537521303662, 0.016788689647763175, 0.1326497848879513, 0.38653678757868315, 0.7392086879874205, 1.0135545312706016, 1.0]
